In [2]:
import pandas as pd
import json

data = pd.read_csv("../dataset/stumbleupon.tsv", sep='\t')
data['title'] = data.boilerplate.map(lambda x: json.loads(x).get('title', ''))
data['body'] = data.boilerplate.map(lambda x: json.loads(x).get('body', ''))
data.head()

,url,urlid,boilerplate,alchemy_category,alchemy_category_score,avglinksize,commonlinkratio_1,commonlinkratio_2,commonlinkratio_3,commonlinkratio_4,...,linkwordscore,news_front_page,non_markup_alphanum_characters,numberOfLinks,numwords_in_url,parametrizedLinkRatio,spelling_errors_ratio,label,title,body
0,http://www.bloomberg.com/news/2010-12-23/ibm-p...,4042,"{""title"":""IBM Sees Holographic Calls Air Breat...",business,0.789131,2.055556,0.676471,0.205882,0.047059,0.023529,...,24,0,5424,170,8,0.152941,0.079130,0,IBM Sees Holographic Calls Air Breathing Batte...,A sign stands outside the International Busine...
1,http://www.popsci.com/technology/article/2012-...,8471,"{""title"":""The Fully Electronic Futuristic Star...",recreation,0.574147,3.677966,0.508021,0.288770,0.213904,0.144385,...,40,0,4973,187,9,0.181818,0.125448,1,The Fully Electronic Futuristic Starting Gun T...,And that can be carried on a plane without the...
2,http://www.menshealth.com/health/flu-fighting-...,1164,"{""title"":""Fruits that Fight the Flu fruits tha...",health,0.996526,2.382883,0.562016,0.321705,0.120155,0.042636,...,55,0,2240,258,11,0.166667,0.057613,1,Fruits that Fight the Flu fruits that fight th...,Apples The most popular source of antioxidants...
3,http://www.dumblittleman.com/2007/12/10-foolpr...,6684,"{""title"":""10 Foolproof Tips for Better Sleep ""...",health,0.801248,1.543103,0.400000,0.100000,0.016667,0.000000,...,24,0,2737,120,5,0.041667,0.100858,1,10 Foolproof Tips for Better Sleep,There was a period in my life when I had a lot...
4,http://bleacherreport.com/articles/1205138-the...,9006,"{""title"":""The 50 Coolest Jerseys You Didn t Kn...",sports,0.719157,2.676471,0.500000,0.222222,0.123457,0.043210,...,14,0,12032,162,10,0.098765,0.082569,0,The 50 Coolest Jerseys You Didn t Know Existed...,Jersey sales is a curious business Whether you...


## Predicting "Greenness" Of Content

This dataset comes from [stumbleupon](https://www.stumbleupon.com/), a web page recommender.  

A description of the columns is below

FieldName|Type|Description
---------|----|-----------
url|string|Url of the webpage to be classified
title|string|Title of the article
body|string|Body text of article
urlid|integer| StumbleUpon's unique identifier for each url
boilerplate|json|Boilerplate text
alchemy_category|string|Alchemy category (per the publicly available Alchemy API found at www.alchemyapi.com)
alchemy_category_score|double|Alchemy category score (per the publicly available Alchemy API found at www.alchemyapi.com)
avglinksize| double|Average number of words in each link
commonlinkratio_1|double|# of links sharing at least 1 word with 1 other links / # of links
commonlinkratio_2|double|# of links sharing at least 1 word with 2 other links / # of links
commonlinkratio_3|double|# of links sharing at least 1 word with 3 other links / # of links
commonlinkratio_4|double|# of links sharing at least 1 word with 4 other links / # of links
compression_ratio|double|Compression achieved on this page via gzip (measure of redundancy)
embed_ratio|double|Count of number of <embed> usage
frameBased|integer (0 or 1)|A page is frame-based (1) if it has no body markup but have a frameset markup
frameTagRatio|double|Ratio of iframe markups over total number of markups
hasDomainLink|integer (0 or 1)|True (1) if it contains an <a> with an url with domain
html_ratio|double|Ratio of tags vs text in the page
image_ratio|double|Ratio of <img> tags vs text in the page
is_news|integer (0 or 1) | True (1) if StumbleUpon's news classifier determines that this webpage is news
lengthyLinkDomain| integer (0 or 1)|True (1) if at least 3 <a> 's text contains more than 30 alphanumeric characters
linkwordscore|double|Percentage of words on the page that are in hyperlink's text
news_front_page| integer (0 or 1)|True (1) if StumbleUpon's news classifier determines that this webpage is front-page news
non_markup_alphanum_characters|integer| Page's text's number of alphanumeric characters
numberOfLinks|integer Number of <a>|markups
numwords_in_url| double|Number of words in url
parametrizedLinkRatio|double|A link is parametrized if it's url contains parameters or has an attached onClick event
spelling_errors_ratio|double|Ratio of words not found in wiki (considered to be a spelling mistake)
label|integer (0 or 1)|User-determined label. Either evergreen (1) or non-evergreen (0); available for train.tsv only

> ### Let's try extracting some of the text content.
> ### Create a feature for the title containing 'recipe'. Is the % of evegreen websites higher or lower on pages that have recipe in the the title?

In [3]:
# Option 1: Create a function to check for this

def has_recipe(text_in):
    try:
        if 'recipe' in str(text_in).lower():
            return 1
        else:
            return 0
    except: 
        return 0
        
data['recipe'] = data['title'].map(has_recipe)

# Option 2: lambda functions

#data['recipe'] = data['title'].map(lambda t: 1 if 'recipe' in str(t).lower() else 0)


# Option 3: string functions
data['recipe'] = data['title'].str.contains('recipe')

 ### Demo: Use of the Count Vectorizer

In [24]:
titles = data['body'].fillna('')

from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(max_features = 1000, 
                             ngram_range=(1, 2), 
                             stop_words='english',
                             binary=True)

# Use `fit` to learn the vocabulary of the titles
vectorizer.fit(titles)

# Use `tranform` to generate the sample X word matrix - one column per feature (word or n-grams)
X = vectorizer.transform(titles)

In [10]:
id2word = dict(enumerate(vectorizer.get_feature_names()))
id2word

{0: u'000',
 1: u'10',
 2: u'10 best',
 3: u'10 things',
 4: u'10 ways',
 5: u'100',
 6: u'101',
 7: u'101 cookbooks',
 8: u'11',
 9: u'12',
 10: u'13',
 11: u'14',
 12: u'15',
 13: u'16',
 14: u'17',
 15: u'18',
 16: u'20',
 17: u'2007',
 18: u'2008',
 19: u'2008 sports',
 20: u'2009',
 21: u'2010',
 22: u'2010 sports',
 23: u'2011',
 24: u'2011 sports',
 25: u'2012',
 26: u'2013',
 27: u'2013 check',
 28: u'2013 sports',
 29: u'22',
 30: u'24',
 31: u'25',
 32: u'30',
 33: u'3d',
 34: u'50',
 35: u'8211',
 36: u'8217',
 37: u'8230',
 38: u'abs',
 39: u'accessories',
 40: u'actually',
 41: u'adventures',
 42: u'advice',
 43: u'air',
 44: u'alcohol',
 45: u'allrecipes',
 46: u'allrecipes com',
 47: u'almond',
 48: u'alton brown',
 49: u'amazing',
 50: u'america',
 51: u'american',
 52: u'analysis',
 53: u'anderson',
 54: u'android',
 55: u'angeles',
 56: u'angeles slideshows',
 57: u'anxiety',
 58: u'apple',
 59: u'apple pie',
 60: u'apples',
 61: u'apps',
 62: u'archive',
 63: u'art',

 ### Demo: Build a random forest model to predict evergreeness of a website using the title features

In [27]:
from gensim.models.ldamodel import LdaModel
from gensim.matutils import Sparse2Corpus
id2word = dict(enumerate(vectorizer.get_feature_names()))
corpus = Sparse2Corpus(X, documents_columns=False)
lda_model = LdaModel(corpus = corpus, id2word=id2word, num_topics=2)

In [28]:
num_topics = 25
num_words_per_topic = 5

for ti, topic in enumerate(lda_model.show_topics(num_topics=num_topics, num_words=num_words_per_topic)):
    print("Topic: %d" % (ti))
    print (topic)
    print()


Topic: 0
(0, u'0.007*"like" + 0.006*"just" + 0.006*"time" + 0.006*"new" + 0.005*"make"')
()
Topic: 1
(1, u'0.007*"recipe" + 0.007*"minutes" + 0.006*"just" + 0.006*"make" + 0.006*"cup"')
()


In [23]:
lda_model.show_topics(num_topics=25)

[(0,
  u'0.045*"foods" + 0.027*"apple" + 0.025*"com" + 0.024*"day" + 0.020*"10" + 0.017*"blog" + 0.017*"shows" + 0.016*"blueberry" + 0.015*"maple" + 0.015*"archive"'),
 (1,
  u'0.055*"com" + 0.053*"life" + 0.041*"sports" + 0.033*"style" + 0.032*"insidershealth com" + 0.032*"insidershealth" + 0.028*"20" + 0.024*"photos" + 0.023*"year" + 0.018*"com insidershealth"'),
 (2,
  u'0.060*"recipe" + 0.040*"cheese" + 0.038*"cupcakes" + 0.024*"homemade" + 0.021*"cream" + 0.019*"cinnamon" + 0.017*"strawberry" + 0.017*"recipes" + 0.016*"cupcake" + 0.015*"perfect"'),
 (3,
  u'0.044*"recipe" + 0.039*"network" + 0.038*"food" + 0.033*"kitchen" + 0.032*"food network" + 0.031*"butter" + 0.030*"pie" + 0.028*"chocolate" + 0.025*"technology" + 0.024*"peanut"'),
 (4,
  u'0.105*"recipe" + 0.057*"com" + 0.039*"allrecipes com" + 0.039*"recipe allrecipes" + 0.039*"allrecipes" + 0.030*"sauce" + 0.017*"baked" + 0.017*"chicken" + 0.016*"woman" + 0.016*"cooks"'),
 (5,
  u'0.058*"best" + 0.032*"world" + 0.026*"photo"

In [4]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(n_estimators = 20)
    
# Use `fit` to learn the vocabulary of the titles
vectorizer.fit(titles)

# Use `tranform` to generate the sample X word matrix - one column per feature (word or n-grams)
X = vectorizer.transform(titles).toarray()
y = data['label']

from sklearn.cross_validation import cross_val_score

scores = cross_val_score(model, X, y, scoring='roc_auc')
print('CV AUC {}, Average AUC {}'.format(scores, scores.mean()))

CV AUC [ 0.78695201  0.80649177  0.80522998], Average AUC 0.799557921281


In [29]:
from gensim.models.word2vec import Word2Vec

text = data.body.dropna().map(lambda x: x.split())
model = Word2Vec(text, size=100, window=5, min_count=5, workers=4)

In [31]:
model.most_similar(positive=['vegan'])
model.

[(u'vegetarian', 0.8475039005279541),
 (u'nutritious', 0.8299712538719177),
 (u'sweets', 0.8202641010284424),
 (u'gourmet', 0.8123573064804077),
 (u'seafood', 0.7819690704345703),
 (u'juicing', 0.7746219635009766),
 (u'healthy,', 0.77190762758255),
 (u'cookies,', 0.7698451280593872),
 (u'cereals', 0.7685773372650146),
 (u'tacos', 0.7648497819900513)]

### Exercise: Build a random forest model to predict evergreeness of a website using the title features and quantitative features

In [5]:
# Use `tranform` to generate the sample X word matrix - one column per feature (word or n-grams)
X_text_features = vectorizer.transform(titles)

# Identify the features you want from the original dataset
other_features_columns = ['html_ratio', 'image_ratio']
other_features = data[other_features_columns]

# Stack them horizontally together
# This takes all of the word/n-gram columns and appends on two more columns for `html_ratio` and `image_ratio`
from scipy.sparse import hstack
X = hstack((X_text_features, other_features)).toarray()

scores = cross_val_score(model, X, y, scoring='roc_auc')
print('CV AUC {}, Average AUC {}'.format(scores, scores.mean()))

# What features of these are most important?
model.fit(X, y)

all_feature_names = vectorizer.get_feature_names() + other_features_columns
feature_importances = pd.DataFrame({'Features' : all_feature_names, 'Importance Score': model.feature_importances_})
feature_importances.sort_values('Importance Score', ascending=False).head()

CV AUC [ 0.78500263  0.79911166  0.79822481], Average AUC 0.794113032767


,Features,Importance Score
1000,html_ratio,0.155060
1001,image_ratio,0.094585
712,recipe,0.035272
721,recipes,0.019651
190,chocolate,0.012699


 ### Exercise: Build a random forest model to predict evergreeness of a website using the body features

In [6]:
body_text = data['body'].fillna('')

# Use `fit` to learn the vocabulary
vectorizer.fit(body_text)

# Use `tranform` to generate the sample X word matrix - one column per feature (word or n-grams)
X = vectorizer.transform(body_text).toarray()

scores = cross_val_score(model, X, y, scoring='roc_auc')
print('CV AUC {}, Average AUC {}'.format(scores, scores.mean()))

CV AUC [ 0.83658603  0.84479776  0.83873979], Average AUC 0.840041195125


 ### Exercise: Use `TfIdfVectorizer` instead of `CountVectorizer` - is this an improvement?

In [7]:
titles = data['title'].fillna('')

from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(max_features = 1000, 
                             ngram_range=(1, 2), 
                             stop_words='english')


# Use `fit` to learn the vocabulary
vectorizer.fit(body_text)

# Use `tranform` to generate the sample X word matrix - one column per feature (word or n-grams)
X = vectorizer.transform(body_text).toarray()

scores = cross_val_score(model, X, y, scoring='roc_auc')
print('CV AUC {}, Average AUC {}'.format(scores, scores.mean()))

CV AUC [ 0.84268957  0.85274835  0.8393846 ], Average AUC 0.844940841904
